In [ ]:
from glob import glob
import os
from pathlib import Path
from PipeLine import *

In [ ]:
from dask.distributed import Client
from dask import delayed
from dask import compute
client = Client(n_workers=4)
client

In [ ]:
database = '/Users/amirhsi_mini/trjs_analysis_segments/'
all_groups = glob(database+"N*-all-analysis-whole_simulations/")

In [ ]:
def ens_avg_all_analyze(group_path, geometry, seperator, parent_path, simulation_type):
    
    group_name = group_path.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis'
    group_path = parent_path+group_name
    Path(group_path).mkdir(parents=True, exist_ok=False)
    group_save_to = group_path+"/"
    
    group_ens_avg_name = group_path.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis-ens_avg'

    group_ens_avg_path = parent_path+group_ens_avg_name
    Path(group_ens_avg_path).mkdir(parents=True, exist_ok=False)
    group_ens_avg_save_to = group_ens_avg_path+"/"
    
    hist_names = ['rHistsCrd','rHistsMon','zHistsCrd','zHistsMon','thetaHistsCrd','thetaHistsMon','rPhisCrd','rPhisMon','rRhosCrd','rRhosMon','zPhisCrd','zPhisMon','zRhosCrd','zRhosMon']
    hist_files = glob(group_path+"/N*/N*.csv")
    
    for hist_name in hist_names:
        ext = [seperator+hist_name+'.csv']
        hist_files = PipeLine.file_reader(hist_files, extensions=ext)
        hist_dicts = PipeLine.ensemble(hist_files, hist_name, geometry, sep=seperator, index_col=0, skiprows=1, save_to=group_save_to)
        ens_evg_dict_rhist = PipeLine.group(hist_dicts, hist_name, geometry, save_to=save_to)

In [ ]:
def ens_avg_all_analyze(group_data, geometry, seperator, parent_path, simulation_type):
    
    group_name = group_data.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis'
    group_path = parent_path+group_name
    Path(group_path).mkdir(parents=True, exist_ok=False)
    group_save_to = group_path+"/"
    
    group_ens_avg_name = group_data.split("/")[-2].split('-')[0]+'-'+simulation_type+'-analysis-ens_avg'
    group_ens_avg_path = parent_path+group_ens_avg_name
    Path(group_ens_avg_path).mkdir(parents=True, exist_ok=False)
    group_ens_avg_save_to = group_ens_avg_path+"/"
    
    hist_names = ['rHistsCrd','rHistsMon','zHistsCrd','zHistsMon','thetaHistsCrd','thetaHistsMon','rPhisCrd','rPhisMon','rRhosCrd','rRhosMon','zPhisCrd','zPhisMon','zRhosCrd','zRhosMon']
    csv_files = glob(group_data+"N*.csv")
    
    for hist_name in hist_names:
        ext = [seperator+hist_name+'.csv']
        hist_files = PipeLine.file_reader(csv_files, extensions=ext)
        hist_dicts = PipeLine.ensemble(hist_files, hist_name, geometry, sep=seperator, index_col=0, skiprows=1, save_to=group_save_to)
        ens_evg_dict_rhist = PipeLine.group(hist_dicts, hist_name, geometry, save_to=group_ens_avg_save_to)

In [ ]:
simulation_type = 'all' # bug or all
geometry = 'cylindrical'
seperator = '-'
input_dir = "trjs_analysis_segments"
parent_dir = "trjs_analysis_all"

analysis_delayed = [] 
for all_group in all_groups:
    parent_path = all_group.split(input_dir)[0]+parent_dir+"/"  
    analysis = delayed(ens_avg_all_analyze)(all_group, geometry, seperator, parent_path, simulation_type)
    analysis_delayed.append(analysis)

In [ ]:
%%time
results = compute(analysis_delayed)